In [1]:
import os
import tempfile
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

plt.style.use('seaborn-whitegrid')

2023-04-24 16:28:49.908603: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/tmp/ipykernel_2016033/1293145838.py:10: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [2]:
import os
import glob
from pymongo import MongoClient
import pandas as pd

database = "vnw_job"
col = "items"

mongo = MongoClient("mongodb://10.122.6.17:27017")
df_meta = pd.DataFrame(list(mongo[database][col].find()))
df_meta['jobTitle'] = df_meta.categoricalProps.apply(lambda x: x["jobTitle"][0])
df_meta['skills'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["skills"]))
df_meta['industries'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["industries"]))
df_meta['locations'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["locations"]))
df_meta['jobLevel'] = df_meta.categoricalProps.apply(lambda x: ','.join(x["jobLevel"]))
df_meta['availableDate'] = df_meta.dateProps.apply(lambda x: x["availableDate"])
df = pd.read_parquet(glob.glob("/home/spark/ylv/data/navidata/*.parquet"))

In [3]:
ratings = df.merge(df_meta, left_on="targetEntityId", right_on="_id", how="left")

In [4]:
ratings = ratings.dropna()

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
rating_1st, rating_2nd = train_test_split(ratings, test_size=0.95)

In [7]:
rating_1st.shape, rating_2nd.shape

((393696, 13), (7480234, 13))

In [8]:
%%time

movies = tf.data.Dataset.from_tensor_slices({
    "job_id": df_meta._id.values.tolist(),
    "category": df_meta.industries.values.tolist(),
    "location": df_meta.locations.values.tolist(),
    "level": df_meta.jobLevel.values.tolist(),
    "job_title": df_meta.jobTitle.values.tolist(),
    "skill_text": df_meta.skills.values.tolist()
})

tensor_slices = {
    "user_id": rating_1st.entityId.values.tolist(),
    "job_id": rating_1st.targetEntityId.values.tolist(),
    "location": rating_1st.locations.values.tolist(),
    "category": rating_1st.industries.values.tolist(),
    "level": rating_1st.jobLevel.values.tolist(),
    "job_title": rating_1st.jobTitle.values.tolist(),
    "skill_text": rating_1st.skills.values.tolist()
}

CPU times: user 427 ms, sys: 301 ms, total: 728 ms
Wall time: 741 ms


2023-04-24 16:31:31.062317: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 16:31:31.089521: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 16:31:31.089675: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 16:31:31.090129: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [9]:
%%time

ds_rating_1st = tf.data.Dataset.from_tensor_slices(tensor_slices)

CPU times: user 2.69 s, sys: 75.5 ms, total: 2.77 s
Wall time: 2.77 s


In [10]:
%%time

from functools import reduce

unique_job_titles = np.unique(df_meta.jobTitle.values.tolist())
unique_job_ids = np.unique(df_meta._id.values.tolist())
unique_user_ids = np.unique(rating_1st.entityId.values.tolist())
unique_locations = np.unique(reduce(lambda x, y: x + "," + y, df_meta.locations.values.tolist()).split(","))
unique_skills = np.unique(reduce(lambda x, y: x + "," + y, df_meta.skills.values.tolist()).split(","))
# job_title_vocabs = np.unique(reduce(lambda x, y: x + y, df_meta.title.str.split(" ").tolist()))
unique_category = np.unique(reduce(lambda x, y: x + "," + y, df_meta.industries.values.tolist()).split(","))

CPU times: user 1.35 s, sys: 87.7 ms, total: 1.44 s
Wall time: 1.44 s


In [11]:
%%time

unique_levels = np.unique(reduce(lambda x, y: x + "," + y, df_meta.jobLevel.values.tolist()).split(","))

CPU times: user 118 ms, sys: 3.7 ms, total: 122 ms
Wall time: 121 ms


In [12]:
class QueryModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(self, layer_sizes):
    """Model for encoding user queries.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None, name='user_stringlookup_layer'),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32, name='user_embedding_layer'),
    ], name='user_embedding_model')

#     # Then construct the layers.
#     self.dense_layers = tf.keras.Sequential()

#     # Use the ReLU activation for all but the last layer.
#     for layer_size in layer_sizes[:-1]:
#       self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

#     # No activation for the last layer.
#     for layer_size in layer_sizes[-1:]:
#       self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    return self.embedding_model(inputs["user_id"])

In [115]:
class CustomTextVectorization(tf.keras.layers.Layer):
    def __init__(self, split, vocabulary, **kwargs):
        super(CustomTextVectorization, self).__init__(**kwargs)
        self.split = split

    def call(self, inputs):
        return tf.keras.layers.TextVectorization(
            max_tokens=100,
            standardize=None,
            split=self.split,
            vocabulary=vocabulary,
            pad_to_max_tokens=True
        )

    def get_config(self):
        config = super(CustomTextVectorization, self).get_config()
        config.update({"split": self.split})
        return config

In [116]:
def tokenization(t):
    return tf.strings.split(t, ',')


def tokenization_1(t):
    return tf.strings.split(t, ' ')


class CandidateModel(tf.keras.Model):
  """Model for encoding movies."""

  def __init__(self, layer_sizes):
    """Model for encoding movies.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    self.job_id_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_job_ids,mask_token=None),
      tf.keras.layers.Embedding(len(unique_job_ids) + 1, 32)
    ])
    self.location_embedding = tf.keras.Sequential([
      CustomTextVectorization(tokenization, unique_locations),
      tf.keras.layers.Embedding(len(unique_locations) + 1, 32),
      tf.keras.layers.GlobalAvgPool1D()
    ])
#     self.level_embedding = tf.keras.Sequential([
#       tf.keras.layers.TextVectorization(
#           max_tokens=100,
#           standardize=None,
#           split=tokenization,
#           vocabulary=unique_levels,
#           pad_to_max_tokens=True
#       ),
#       tf.keras.layers.Embedding(len(unique_levels) + 1, 32),
#       tf.keras.layers.GlobalAvgPool1D()
#     ])
#     self.category_embedding = tf.keras.Sequential([
#       tf.keras.layers.TextVectorization(
#           max_tokens=100,
#           standardize=None,
#           split=tokenization,
#           vocabulary=unique_category,
#           pad_to_max_tokens=True
#       ),
#       tf.keras.layers.Embedding(len(unique_category) + 1, 64),
#       tf.keras.layers.GlobalAvgPool1D()
#     ])
#     self.skill_embedding = tf.keras.Sequential([
#       tf.keras.layers.TextVectorization(
#           max_tokens=100,
#           standardize=None,
#           split=tokenization,
#           vocabulary=unique_skills,
#           pad_to_max_tokens=True
#       ),
#       tf.keras.layers.Embedding(len(unique_skills) + 1, 32),
#       tf.keras.layers.GlobalAvgPool1D()
#     ])
#     self.embedding_model = MovieModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = tf.concat([
        self.job_id_embedding(inputs["job_id"]),
#         self.category_embedding(inputs["category"]),
        self.location_embedding(inputs["location"]),
#         self.level_embedding(inputs["level"]),
#         self.skill_embedding(features["skill_text"])
    ], axis=1)
    return self.dense_layers(feature_embedding)

In [117]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.query_model: tf.keras.Model = user_model
    self.candidate_model: tf.keras.Model = movie_model
    self.task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"]
    })
    movie_embeddings = self.candidate_model({
        "job_id": features["job_id"],
        "job_title": features["job_title"],
        "category": features["category"],
        "location": features["location"],
        "skill_text": features["skill_text"],
        "level": features["level"],
    })

    return self.task(
        query_embeddings, movie_embeddings, compute_metrics=not training)

In [118]:
cached_train_1st = ds_rating_1st.batch(4096)

In [119]:
%%time

num_epochs = 1

user_model = QueryModel([128, 64, 32])
item_model = CandidateModel([128, 64, 32])

CPU times: user 126 ms, sys: 24.3 ms, total: 150 ms
Wall time: 149 ms


In [122]:
config = item_model.get_config()
custom_objects = {'split': tokenization, 'CustomTextVectorization': CustomTextVectorization}
with tf.keras.utils.custom_object_scope(custom_objects):
    new_item_model = tf.keras.Model.from_config(config)

In [123]:
model = MovielensModel(user_model, new_item_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

NotImplementedError: Exception encountered when calling layer 'model_2' (type Model).

Unimplemented `tf.keras.Model.call()`: if you intend to create a `Model` with the Functional API, please provide `inputs` and `outputs` arguments. Otherwise, subclass `Model` with an overridden `call()` method.

Call arguments received by layer 'model_2' (type Model):
  • inputs={'job_id': 'tf.Tensor(shape=(None,), dtype=string)', 'category': 'tf.Tensor(shape=(None,), dtype=string)', 'location': 'tf.Tensor(shape=(None,), dtype=string)', 'level': 'tf.Tensor(shape=(None,), dtype=string)', 'job_title': 'tf.Tensor(shape=(None,), dtype=string)', 'skill_text': 'tf.Tensor(shape=(None,), dtype=string)'}
  • training=None
  • mask=None

In [93]:
history = model.fit(
    cached_train_1st,
    validation_freq=5,
    epochs=num_epochs,
    verbose=0
)

In [38]:
model.query_model.save("navi_model/test_user_model_1")

INFO:tensorflow:Assets written to: navi_model/test_user_model_1/assets


In [81]:
model.candidate_model.save("navi_model/test_item_model_1")

INFO:tensorflow:Assets written to: navi_model/test_item_model_1/assets


In [41]:
model.query_model.summary()

Model: "query_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 user_embedding_model (Seque  (None, 32)               5662784   
 ntial)                                                          
                                                                 
Total params: 5,662,784
Trainable params: 5,662,784
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.candidate_model.summary()

Model: "candidate_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_15 (Sequential)  (None, 32)                613888    
                                                                 
 sequential_16 (Sequential)  (None, 32)                2080      
                                                                 
 sequential_17 (Sequential)  (None, 32)                192       
                                                                 
 sequential_18 (Sequential)  (None, 64)                4672      
                                                                 
 sequential_19 (Sequential)  (None, 32)                30944     
                                                                 
Total params: 651,776
Trainable params: 651,776
Non-trainable params: 0
_________________________________________________________________


In [82]:
tf.keras.models.load_model("navi_model/test_item_model_1")

ValueError: Unkown value for `split` argument of layer TextVectorization. If restoring a model and `split` is a custom callable, please ensure the callable is registered as a custom object. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details. Allowed values are: `None`, a `Callable`, or one of the following values: ('whitespace', 'character'). Received: tokenization